### Libraries and Modules
[ORCA Library for Python](https://github.com/sybrenstuvel/Python-RVO2)

[RVO2 Library Documentation](https://gamma.cs.unc.edu/RVO2/documentation/2.0/)

In [48]:
import rvo2
import math
from csv import writer
import time

import numpy as np
import import_ipynb
import voronoi_constructor as bvc_builder

### Global Parameters

In [37]:
time_step = 1
max_speed = 1
radius = 0.1
time_horizon = 20
time_horizon_obst = 0
velocity = (1,1)
for i in range(max_neighbors+1):
    print('Initial position:',init_pos[i])
    print('Final position:',final_pos[i])

Initial position: [29.4 -1.4]
Final position: [-33.6  -2.3]
Initial position: [ 6.9 28.3]
Final position: [-12.6 -30.6]
Initial position: [-25.8  15.5]
Final position: [ 24.5 -20.9]
Initial position: [-27.7 -19. ]
Final position: [20.9 14. ]
Initial position: [  4.9 -30.9]
Final position: [-10.9  29.4]


### Main Loop

In [76]:
def main_module(num_robots, seed, csv_writer):
    start_time = time.time()

    max_neighbors = num_robots-1
    neighbor_dist = math.sqrt(2*num_robots)
    init_pos, final_pos, dist = bvc_builder.position_generator(num_robots,
                                                               seed)

    sim = rvo2.PyRVOSimulator(time_step, neighbor_dist, max_neighbors,
                              time_horizon, time_horizon_obst, radius,
                              max_neighbors)
    agents = []
    checkered_flags = []
    inst_distance = [0 for _ in range(num_robots)]
    for i in range(max_neighbors+1):
        agents.append(sim.addAgent(tuple(init_pos[i])))
        inst = final_pos[i] - init_pos[i]
        sim.setAgentPrefVelocity(agents[i], tuple(inst/np.linalg.norm(inst)))
        checkered_flags.append(False)
    checkered_flags = np.array(checkered_flags)

    #print('Simulation has %i agents.' % (sim.getNumAgents()))
    #print('Running simulation!\n')

    step = 0
    while not np.all(checkered_flags):
        sim.doStep()
        positions_str = []
        positions = []
        for i, agent_no in enumerate(agents):
            positions_str.append('(%5.3f, %5.3f)' % sim.getAgentPosition(agent_no))
            positions.append(sim.getAgentPosition(agent_no))
            traveled_distance = np.linalg.norm(np.array(positions[i]) - init_pos[i])
            if traveled_distance < dist[i]:
                if step > 0:
                    inst_distance[i] += np.linalg.norm(np.array(positions[i])
                                                       - np.array(positions_prev[i]))
            elif not checkered_flags[i]:
                checkered_flags[i] = True
                inst_distance[i] += np.linalg.norm(final_pos[i]
                                                - np.array(positions[i]))
        step += 1
        positions_prev = positions
    csv_writer.writerow([num_robots, seed, step, time.time() - start_time,
                         np.sum(dist) / np.sum(inst_distance)])

### Experiments

In [77]:
with open("/home/leduin/Desktop/Hub/Results/orca.csv","a") as file:
    csv_writer = writer(file)
    for i in range(10):
        num_robots = 5+i
        for seed in range(30):
            main_module(num_robots,seed,csv_writer)